In [18]:
import pandas as pd
import numpy as np
import re
import string

In [19]:
train = pd.read_csv('train.csv', low_memory=False)
test = pd.read_csv('test.csv', low_memory=False)

In [20]:
test['target'] = 2
totaldata = pd.concat([train,test])
totaldata['textsize'] = totaldata['text'].apply(len)
totaldata['punctuation_count'] = totaldata['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
totaldata['word_mean_len'] = totaldata['text'].apply(lambda x: np.mean([len(c) for c in str(x).split()])) 
totaldata['haslocacion'] = totaldata['location'].notna()
totaldata.drop('location',axis=1,inplace=True)

In [21]:
############## keyword

In [22]:
keyword = pd.read_csv('data_process/keyword.csv')
totaldata = totaldata.merge(keyword,how='left')
totaldata.drop('keyword',axis=1,inplace=True)
del keyword

In [23]:
############## links, label, mentions

In [24]:
def get_links(x):
    l = re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',x)
    return l
def hashtags(x): 
    return re.findall(r'#[a-zA-z0-9_]{1,}',x)
def arrobas(x):
    return re.findall(r'@[a-zA-z0-9_]{1,}',x)

totaldata[["mentions_count","links_count","label_count"]] =\
                        totaldata["text"].apply(lambda x: pd.Series([len(arrobas(x)),len(hashtags(x)),len(get_links(x))]))

In [25]:
############# dominos 

In [26]:
enlaces_target = pd.read_csv('data_process/enlaces.csv')
totaldata = totaldata.merge(enlaces_target,how='left').fillna(0)
del enlaces_target

In [27]:
############# taggs

In [28]:
tagged=pd.read_csv('data_process/tagged.csv')
totaldata = pd.concat([totaldata, tagged],axis=1)

In [29]:
totaldata.to_csv('data_process/totaldata.csv',index=False)

In [30]:
totaldata.columns

Index(['id', 'text', 'target', 'textsize', 'punctuation_count',
       'word_mean_len', 'haslocacion', 'keyword_t_count', 'keyword_f_count',
       'keyword_metric', 'mentions_count', 'links_count', 'label_count',
       'domain_metric_mean', 'domain_metric_std', 'domain_metric_max',
       'domain_metric_min', 'PRP$', 'NNS', 'VBP', 'DT', 'NNP', 'IN', 'NN',
       'PRP', 'VBD', 'TO', 'VB', 'VBG', 'VBN', 'JJ', 'CC', 'RB', 'VBZ', 'MD',
       'EX', 'CD', 'WP', 'RP', 'NNPS', 'JJR', 'WRB', 'JJS', 'WDT', 'RBR',
       'RBS', 'FW', 'PDT', 'POS', 'UH', 'SYM', 'WP$', ''''],
      dtype='object')